# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress
from itertools import islice 
import math

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
print(cities)

['busselton', 'aklavik', 'bluff', 'klaksvik', 'pangnirtung', 'rikitea', 'hermanus', 'yellowknife', 'vaini', 'priiskovyy', 'castro', 'dikson', 'zhigansk', 'mosjoen', 'thompson', 'souillac', 'xuddur', 'nishihara', 'ushuaia', 'camacha', 'vestmannaeyjar', 'kresttsy', 'tiksi', 'san quintin', 'mataura', 'georgetown', 'mahebourg', 'cayenne', 'rawson', 'tabou', 'khatanga', 'tevaitoa', 'kaeo', 'alofi', 'butaritari', 'pamanukan', 'airai', 'saint-francois', 'mar del plata', 'coquimbo', 'tilichiki', 'yar-sale', 'morondava', 'savinskiy', 'utete', 'chacabuco', 'albany', 'tomatlan', 'punta arenas', 'cape town', 'port blair', 'atuona', 'bambous virieux', 'fairbanks', 'vysokogornyy', 'fortuna', 'xuanhua', 'barrow', 'bethel', 'taolanaro', 'bredasdorp', 'villa oliva', 'longyearbyen', 'cherskiy', 'hilo', 'bereznik', 'bandarbeyla', 'sao filipe', 'rocha', 'upernavik', 'palmer', 'ostrovnoy', 'oxford', 'hami', 'alekseyevsk', 'avarua', 'korla', 'ushtobe', 'sao jose da coroa grande', 'kapaa', 'sagua la grande',

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [16]:
# Define function to create a dictionary containing weather data 
# for a single city, and append it to the dictionary.
def create_city_dict(city):
    # Create a list to hold weather data for a single city
    city_weather = []

    # Save config information
    url = "http://api.openweathermap.org/data/2.5/weather?"

    # Build query URL
    query_url = f"{url}appid={weather_api_key}&q={city}&units=Imperial"

    # Read JSON data to call the Open Weather Map API.
    weather_response = requests.get(query_url)
    weather_json = weather_response.json()

    # Set variables to hold relevant data
    city_name = weather_json["name"]
    lat = weather_json["coord"]["lat"]
    lon = weather_json["coord"]["lon"]
    max_temp = weather_json["main"]["temp_max"]
    cloud = weather_json["clouds"]["all"]
    wind_speed = weather_json["wind"]["speed"]
    humidity = weather_json["main"]["humidity"]
    date = weather_json["dt"]
    id_code = weather_json["id"]
    country = weather_json["sys"]["country"]

    # Append variables to list
    city_weather.append(city_name)
    city_weather.append(cloud)
    city_weather.append(country)
    city_weather.append(date)
    city_weather.append(humidity)
    city_weather.append(lat)
    city_weather.append(lon)
    city_weather.append(max_temp)
    city_weather.append(wind_speed)

    # Update all_cities_weather dictionary to include data for that city
   
    return city_weather

In [20]:
# Create a dictionary to hold each city name as a key
# and a list of city weather data as values
all_cities_weather = {}

# Perform a loop of API calls cities
try:
    # Loop through x groups of 50 cities.  
    # If number of cities is not divisible by 50, 
    # then the last group will raise an IndexError. 
    # Raise an Exception IndexError to Terminate the loop.
    for x in range(0, math.floor(len(cities)/50+1)):
        try:
            # Loop through each set of 50 cities
            for y in range(0, 49):
                
                # Create a variable to hold weather data for that city.
                city = create_city_dict(cities[x*50+y])   
    
                # Update all_cities_weather dictionary to include data for that city
                all_cities_weather.update({cities[x*50+y]:city})
    
                # Print index number of city, index number of groups, and name of city
                print(f"Processing Record {y+1} of Set {x+1} | {cities[x*50+y]}")
        
        except KeyError:
            print("City not found. Skipping...")

    # Delay 20 seconds between API calls
    time.sleep(20)
    
    # Exception IndexError to terminate the loop.
except IndexError:
    print(f"------------------------------") 
    print(f"Data Retrieval Complete") 
    print(f"------------------------------")

print(all_cities_weather)

Processing Record 1 of Set 1 | busselton
Processing Record 2 of Set 1 | aklavik
Processing Record 3 of Set 1 | bluff
Processing Record 4 of Set 1 | klaksvik
Processing Record 5 of Set 1 | pangnirtung
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | hermanus
Processing Record 8 of Set 1 | yellowknife
Processing Record 9 of Set 1 | vaini
Processing Record 10 of Set 1 | priiskovyy
Processing Record 11 of Set 1 | castro
Processing Record 12 of Set 1 | dikson
Processing Record 13 of Set 1 | zhigansk
Processing Record 14 of Set 1 | mosjoen
Processing Record 15 of Set 1 | thompson
Processing Record 16 of Set 1 | souillac
Processing Record 17 of Set 1 | xuddur
Processing Record 18 of Set 1 | nishihara
Processing Record 19 of Set 1 | ushuaia
Processing Record 20 of Set 1 | camacha
Processing Record 1 of Set 2 | port blair
Processing Record 2 of Set 2 | atuona
Processing Record 3 of Set 2 | bambous virieux
Processing Record 4 of Set 2 | fairbanks
Processing Record 5 of Set 2

In [13]:
print(json.dumps(weather_json, indent=4))

NameError: name 'weather_json' is not defined

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
Summary = pd.DataFrame.from_dict(all_cities_weather, orient="index")

Summary = Summary.rename(columns=
                        {0:"City",
                        1:"Cloudiness",
                        2:"Country",
                        3:"Date",
                        4:"Humidity",
                        5:"Lat",
                        6:"Lng",
                        7:"Max Temp",
                        8:"Wind Speed"}
                        )
Summary
Summary.reset_index(drop=True)

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots


In [ ]:
# Create Northern and Southern Hemisphere DataFrames


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression